In [1]:
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
Movement = pd.read_csv("ArtistMovements.csv")
Artwork = pd.read_csv("Artwork.csv")
Special = pd.read_csv("ArtistSpecializations.csv")
Generated = pd.read_csv("Generated.csv")
Artist = pd.read_csv("Artist.csv")
Education = pd.read_csv("ArtistEducation.csv")
Places = pd.read_csv("Places.csv")
OutputReal = pd.read_csv("output_real.csv")
OutputFake = pd.read_csv("output_fake.csv")
FeaturesFake = pd.read_csv("image_features_fake.csv")
FeaturesReal = pd.read_csv("image_features_real.csv")
Rec = pd.read_csv("n_relations_recommendations.csv")

In [108]:
Education.head(10)

,artist_id,academy_id
0,0.0,0.0
1,0.0,1.0
2,2.0,2.0
3,2.0,3.0
4,9.0,4.0
5,13.0,5.0
6,19.0,6.0
7,19.0,2.0
8,26.0,7.0
9,27.0,8.0


# Artwork

In [128]:
Artwork.shape

(10517, 10)

In [120]:
pd.set_option('max_colwidth', 50)
Artwork.head(3)

,id,name,url,image_url,artist,rating,summary,year,medium,location
0,0,Cafe Terrace on the Place du Forum,http://wikigallery.org/wiki/painting_133032/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,Café Terrace at Night is an 1888 oil painting ...,1888.0,0.0,Kröller-Müller Museum
1,1,Starry Night,http://wikigallery.org/wiki/painting_9698/Vinc...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,NaN,NaN,NaN,NaN
2,2,A Digger,http://wikigallery.org/wiki/painting_132953/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,NaN,NaN,NaN,NaN


In [121]:
Artwork["year"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9751830369877341

In [111]:
Artwork["medium"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9499857373775792

In [112]:
Artwork["location"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9395264809356281

In [17]:
Artwork2 = Artwork.iloc[:, [0,1,3,4,5]]
Artwork2 = Artwork2.rename(columns = {'artist':'artist_id'})
Artwork2.sample(5)

,id,name,image_url,artist_id,rating
818,818,Oannès (Christ et Serpent),https://kuleuven-datathon-2023.s3.eu-central-1...,32,10.0
6984,6985,Kittens at Play,https://kuleuven-datathon-2023.s3.eu-central-1...,397,9.0
2987,2987,The Pyramids and the Sphinx of Gizeh,https://kuleuven-datathon-2023.s3.eu-central-1...,134,7.0
8694,8695,View from the Pass of Sakrigali Looking East T...,https://kuleuven-datathon-2023.s3.eu-central-1...,493,NaN
1266,1266,The Smoke Signal,https://kuleuven-datathon-2023.s3.eu-central-1...,50,NaN


In [18]:
Artwork2["AI_Generated"] = 0
Artwork2.head()

,id,name,image_url,artist_id,rating,AI_Generated
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,0
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,0
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1...,0,6.0,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0


In [116]:
pd.set_option('max_colwidth', 200)
Artwork["image_url"].head(5)

0    https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg
1                          https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg
2                              https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg
3                   https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg
4                       https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg
Name: image_url, dtype: object

# Generated

In [126]:
Generated.shape

(3447, 2)

In [154]:
Generated.head()

,source_artwork,url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1...,1
1,1480,https://kuleuven-datathon-2023.s3.eu-central-1...,1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1...,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1...,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1...,1


In [19]:
Generated["AI_Generated"] = 1 
Generated2 = Generated.rename(columns = {'url':'image_url'})
Generated2.head()

,source_artwork,image_url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1...,1
1,1480,https://kuleuven-datathon-2023.s3.eu-central-1...,1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1...,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1...,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1...,1


# Movement

In [5]:
Movement.head(15)

,artist_id,movement_id
0,0.0,43.0
1,1.0,44.0
2,2.0,44.0
3,4.0,10.0
4,4.0,52.0
5,6.0,32.0
6,6.0,25.0
7,8.0,4.0
8,8.0,14.0
9,8.0,37.0


In [3]:
Movement.shape

(255, 2)

# Specialization 

In [168]:
Special.head()

,artist_id,specialty_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,1.0,2.0


# Artist

In [205]:
Artist.sample(5)

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
265,265,Frank Gresley,https://www.wikigallery.org//wiki/artist61226/...,Frank Gresley (1855–1936) was a British artist...,NaN,NaN,NaN,NaN,NaN,NaN
116,116,Auguste Joseph Marie De Mersseman,https://www.wikigallery.org//wiki/artist57946/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,52,William Merritt Chase,http://wikigallery.org/wiki/artist40281/Willia...,"William Merritt Chase (November 1, 1849 – Octo...",52.0,35.0,142.0,1849-11-01,1916-10-25,NaN
486,486,Myles Birket Foster,https://www.wikigallery.org//wiki/artist48895/...,Myles Birket Foster (4 February 1825 – 27 Mar...,NaN,NaN,NaN,NaN,NaN,NaN
326,326,French School,https://www.wikigallery.org//wiki/artist61277/...,Education in France is organized in a highly c...,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
Artist["cause_of_death"].isna().sum()/len(Artist) #cause of data column should be removed due to large number of NAs

0.9967532467532467

In [208]:
Artist["birthplace"].isna().sum()/len(Artist)

0.5

In [209]:
Artist["deathplace"].isna().sum()/len(Artist)

0.5275974025974026

In [210]:
Artist["birthdate"].isna().sum()/len(Artist)

0.6347402597402597

In [211]:
Artist["deathdate"].isna().sum()/len(Artist)

0.5616883116883117

In [20]:
Artist2 = Artist.iloc[:,[0,4,5,6,7,8]]
Artist2 = Artist2.rename(columns = {"id": "artist_id"})
Artist2['artist_id'] = Artist2['artist_id'].astype(float)
Artist2.head()

,artist_id,picture,birthplace,deathplace,birthdate,deathdate
0,0.0,0.0,0.0,342.0,1853-03-30,1890-07-29
1,1.0,1.0,1.0,343.0,1841-02-25,1919-12-03
2,2.0,2.0,2.0,344.0,1840-11-14,1926-12-05
3,3.0,3.0,3.0,NaN,1932-04-19,NaN
4,4.0,4.0,4.0,2.0,1824-05-11,1904-01-10


# Artist Education

In [223]:
Education.head()

,artist_id,academy_id
0,0.0,0.0
1,0.0,1.0
2,2.0,2.0
3,2.0,3.0
4,9.0,4.0


In [112]:
Education2 = Education.pivot_table(index = "artist_id", columns = "academy_id", aggfunc = "size", fill_value = 0)
Education2

academy_id,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0
artist_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
557.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
572.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [113]:
Education2 = Education2.add_prefix("Education").reset_index(level=0)
Education2

academy_id,artist_id,Education0.0,Education1.0,Education2.0,Education3.0,Education4.0,Education5.0,Education6.0,Education7.0,Education8.0,Education9.0,Education10.0,Education11.0,Education12.0,Education13.0,Education14.0,Education15.0,Education16.0,Education17.0,Education18.0,Education19.0,Education20.0,Education21.0,Education22.0,Education23.0,Education24.0,Education25.0,Education26.0,Education27.0,Education28.0,Education29.0,Education30.0,Education31.0,Education32.0,Education33.0,Education34.0,Education35.0,Education36.0,Education37.0,Education38.0,Education39.0,Education40.0,Education41.0,Education42.0,Education43.0,Education44.0,Education45.0,Education46.0,Education47.0
0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,13.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,545.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58,557.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
59,572.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60,604.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Places

In [229]:
Places.head()

,id,name,parent
0,0,Zundert,213.0
1,1,Limoges,214.0
2,2,Paris,214.0
3,3,Medellín,215.0
4,4,Vesoul,216.0


In [30]:
Places2 = Places.iloc[:,[0,2]]
Places2 = Places2.rename(columns = {"id": "artist_id"})

# Recommendation

In [30]:
Rec.head()

,id,num_rels
0,0,1
1,1,1
2,2,5
3,3,0
4,4,0


# Text split

In [21]:
def FindNumber(inputstr):
    return inputstr.split("\\")[-1].split(".")[0]

In [22]:
OutputReal["filename"] = OutputReal.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [23]:
OutputReal2 = OutputReal.rename(columns = {"filename":"id", "logit": "real_logit", "time":"real_time"})

In [24]:
OutputReal2['id'] = OutputReal2['id'].astype(float) #change to float values to match main dataframe

In [25]:
OutputFake["filename"] = OutputFake.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [26]:
OutputFake2 = OutputFake.rename(columns = {"filename":"id", "logit": "fake_logit", "time":"fake_time"})

In [27]:
OutputFake2['id'] = OutputFake2['id'].astype(float) #change to float values to match main dataframe

# Merge Dataframes

In [28]:
df_list1 = [Artwork2, Generated2]
df1 = pd.concat(df_list1)

In [98]:
df1.head()

,id,name,image_url,artist_id,rating,source_artwork,AI_Generated
0,0.0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0.0,8.0,NaN,NaN
1,1.0,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0.0,9.0,NaN,NaN
2,2.0,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0.0,7.0,NaN,NaN
3,3.0,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0.0,6.0,NaN,NaN
4,4.0,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0.0,8.0,NaN,NaN


In [6]:
Movement["artist_id"]

0        0.0
1        1.0
2        2.0
3        4.0
4        4.0
       ...  
250    604.0
251    606.0
252    609.0
253    610.0
254    611.0
Name: artist_id, Length: 255, dtype: float64

In [8]:
pd.set_option('display.max_columns', None)
Movement_dummies = pd.get_dummies(Movement['movement_id'])
Movement_dummies.columns = Movement_dummies.columns.astype(int).astype(str)

In [9]:
Movement_dummies = Movement_dummies.add_prefix("Movement")
Movement_dummies

,Movement0,Movement1,Movement2,Movement3,Movement4,Movement5,Movement6,Movement7,Movement8,Movement9,Movement10,Movement11,Movement12,Movement13,Movement14,Movement15,Movement16,Movement17,Movement18,Movement19,Movement20,Movement21,Movement22,Movement23,Movement24,Movement25,Movement26,Movement27,Movement29,Movement30,Movement31,Movement32,Movement33,Movement34,Movement35,Movement36,Movement37,Movement38,Movement39,Movement40,Movement41,Movement42,Movement43,Movement44,Movement45,Movement46,Movement47,Movement48,Movement49,Movement50,Movement51,Movement52,Movement53,Movement54
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
251,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
252,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
253,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
Movement2 = pd.concat([Movement["artist_id"],Movement_dummies], axis = 1)
Movement2.head(10)

,artist_id,Movement0,Movement1,Movement2,Movement3,Movement4,Movement5,Movement6,Movement7,Movement8,Movement9,Movement10,Movement11,Movement12,Movement13,Movement14,Movement15,Movement16,Movement17,Movement18,Movement19,Movement20,Movement21,Movement22,Movement23,Movement24,Movement25,Movement26,Movement27,Movement29,Movement30,Movement31,Movement32,Movement33,Movement34,Movement35,Movement36,Movement37,Movement38,Movement39,Movement40,Movement41,Movement42,Movement43,Movement44,Movement45,Movement46,Movement47,Movement48,Movement49,Movement50,Movement51,Movement52,Movement53,Movement54
0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,4.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
type(Movement2)

pandas.core.frame.DataFrame

In [46]:
Special.head()

,artist_id,specialty_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,1.0,2.0


In [81]:
type(Special2)

pandas.core.frame.DataFrame

In [93]:
Special2 = Special.pivot_table(index = "artist_id", columns = "specialty_id", aggfunc = "size", fill_value = 0)
Special2.head()

specialty_id,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0
artist_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4.0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
Special2 = Special2.add_prefix("Special").reset_index(level=0)
Special2.head()

specialty_id,artist_id,Special0.0,Special1.0,Special2.0,Special3.0,Special4.0,Special5.0,Special6.0,Special7.0,Special8.0,Special9.0,Special10.0,Special11.0,Special12.0,Special13.0,Special14.0,Special15.0,Special16.0,Special17.0,Special18.0,Special19.0,Special20.0,Special21.0,Special22.0,Special23.0,Special24.0,Special25.0,Special26.0,Special27.0,Special28.0,Special29.0,Special30.0,Special31.0,Special32.0,Special33.0,Special34.0,Special35.0
0,0.0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
#df_list2 = [Movement, Special, df1, Artist2, Education, Places2]
df_list21 = [df1, Artist2, Education, Places2, Movement2, Special2, Education2]

#df2 = reduce(lambda left, right: pd.merge(left, right, on = ["artist_id"], how = "outer"), df_list2)
df21 = reduce(lambda left, right: pd.merge(left, right, on = ["artist_id"], how = "outer"), df_list21)

In [98]:
df21.head(10)

,id,name,image_url,artist_id,rating,AI_Generated,source_artwork,picture,birthplace,deathplace,birthdate,deathdate,academy_id,parent,Movement0,Movement1,Movement2,Movement3,Movement4,Movement5,Movement6,Movement7,Movement8,Movement9,Movement10,Movement11,Movement12,Movement13,Movement14,Movement15,Movement16,Movement17,Movement18,Movement19,Movement20,Movement21,Movement22,Movement23,Movement24,Movement25,Movement26,Movement27,Movement29,Movement30,Movement31,Movement32,Movement33,Movement34,Movement35,Movement36,Movement37,Movement38,Movement39,Movement40,Movement41,Movement42,Movement43,Movement44,Movement45,Movement46,Movement47,Movement48,Movement49,Movement50,Movement51,Movement52,Movement53,Movement54,Special0.0,Special1.0,Special2.0,Special3.0,Special4.0,Special5.0,Special6.0,Special7.0,Special8.0,Special9.0,Special10.0,Special11.0,Special12.0,Special13.0,Special14.0,Special15.0,Special16.0,Special17.0,Special18.0,Special19.0,Special20.0,Special21.0,Special22.0,Special23.0,Special24.0,Special25.0,Special26.0,Special27.0,Special28.0,Special29.0,Special30.0,Special31.0,Special32.0,Special33.0,Special34.0,Special35.0
0,0.0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,8.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,0.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,8.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,1.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,9.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,0.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,9.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,1.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,7.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,0.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,7.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,1.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,

In [115]:
df_list3 = [df21, OutputReal2, OutputFake2, FeaturesFake, FeaturesReal, Rec]
#df_list4 = [OutputReal2, OutputFake2, FeaturesFake, FeaturesReal]

final_df = reduce(lambda left, right: pd.merge(left, right, on = ["id"], how = "outer"), df_list3)
#features_df = reduce(lambda left, right: pd.merge(left, right, on = ["id"], how = "outer"), df_list4)

In [116]:
final_df.head(5)

,id,name,image_url,artist_id,rating,AI_Generated,source_artwork,picture,birthplace,deathplace,birthdate,deathdate,academy_id,parent,Movement0,Movement1,Movement2,Movement3,Movement4,Movement5,Movement6,Movement7,Movement8,Movement9,Movement10,Movement11,Movement12,Movement13,Movement14,Movement15,Movement16,Movement17,Movement18,Movement19,Movement20,Movement21,Movement22,Movement23,Movement24,Movement25,Movement26,Movement27,Movement29,Movement30,Movement31,Movement32,Movement33,Movement34,Movement35,Movement36,Movement37,Movement38,Movement39,Movement40,Movement41,Movement42,Movement43,Movement44,Movement45,Movement46,Movement47,Movement48,Movement49,Movement50,Movement51,Movement52,Movement53,Movement54,Special0.0,Special1.0,Special2.0,Special3.0,Special4.0,Special5.0,Special6.0,Special7.0,Special8.0,Special9.0,Special10.0,Special11.0,Special12.0,Special13.0,Special14.0,Special15.0,Special16.0,Special17.0,Special18.0,Special19.0,Special20.0,Special21.0,Special22.0,Special23.0,Special24.0,Special25.0,Special26.0,Special27.0,Special28.0,Special29.0,Special30.0,Special31.0,Special32.0,Special33.0,Special34.0,Special35.0,Education0.0,Education1.0,Education2.0,Education3.0,Education4.0,Education5.0,Education6.0,Education7.0,Education8.0,Education9.0,Education10.0,Education11.0,Education12.0,Education13.0,Education14.0,Education15.0,Education16.0,Education17.0,Education18.0,Education19.0,Education20.0,Education21.0,Education22.0,Education23.0,Education24.0,Education25.0,Education26.0,Education27.0,Education28.0,Education29.0,Education30.0,Education31.0,Education32.0,Education33.0,Education34.0,Education35.0,Education36.0,Education37.0,Education38.0,Education39.0,Education40.0,Education41.0,Education42.0,Education43.0,Education44.0,Education45.0,Education46.0,Education47.0,real_logit,real_time,fake_logit,fake_time,edge_score_x,contrast_score_x,brightness_score_x,saturation_score_x,edge_score_y,contrast_score_y,brightness_score_y,saturation_score_y,num_rels
0,0.0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,8.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,0.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.378802,6.175854,NaN,NaN,NaN,NaN,NaN,NaN,46.383220,55.552072,112.148305,127.663731,1.0
1,0.0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,8.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,1.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.378802,6.175854,NaN,NaN,NaN,NaN,NaN,NaN,46.383220,55.552072,112.148305,127.663731,1.0
2,1.0,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0.0,9.0,0.0,NaN,0.0,0.0,342.0,1853-03-30,1890-07-29,0.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [101]:
final_df.to_csv("working_merged_df.csv")

In [24]:
features_df.to_csv("featues_df_final.csv")

NameError: name 'rownames' is not defined

In [40]:
final_df.columns

Index(['artist_id', 'movement_id', 'specialty_id', 'id', 'name', 'image_url',
       'rating', 'source_artwork', 'AI_Generated', 'picture', 'birthplace',
       'deathplace', 'birthdate', 'deathdate', 'academy_id', 'parent',
       'real_logit', 'real_time', 'fake_logit', 'fake_time', 'edge_score_x',
       'contrast_score_x', 'brightness_score_x', 'saturation_score_x',
       'edge_score_y', 'contrast_score_y', 'brightness_score_y',
       'saturation_score_y', 'num_rels'],
      dtype='object')